In [1]:
import analysis3 as a3

In [2]:
## For Aut1367:
points_path = 'Aut1367reorient_atlaslocaleq.csv'

In [3]:
p, uniq = a3.get_regions(points_path, "atlas/ara3_annotation.nii", "points/" + 'Aut1367' + "_regions.csv");

10000
num unique regions: 31
[0, 771, 7, 781, 656, 1041, 1049, 153, 665, 794, 412, 413, 159, 288, 934, 647, 556, 698, 188, 703, 448, 1089, 328, 841, 728, 217, 96, 354, 1123, 996, 101]


analysis3.py:468: VisibleDeprecationWarning:

using a non-integer number instead of an integer will result in an error in the future



In [34]:
import numpy as np
import os

import plotly
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly import tools
import csv
import seaborn as sns

import requests
import json

In [5]:
# Type in the path to your csv file here
thedata = None
thedata = np.genfromtxt('points/Aut1367_regions.csv',
    delimiter=',', dtype='int', usecols = (0,1,2,4), names=['a','b','c', 'region'])

In [6]:
# Sort the csv file by the last column (by order = 'region') using ndarray.sort
sort = np.sort(thedata, order = 'region');

In [7]:
# Save a copy of this sorted csv in the points folder
if not os.path.exists('points'):
    os.makedirs('points')
np.savetxt('points/Aut1367_regions_sorted.csv', sort, fmt='%d', delimiter=',')

In [21]:
# Find the centroids of each region

sorted_regions = np.sort(uniq);

current_region = sorted_regions[0];
i = 0;
x = [];
y = [];
z = [];
centroids = {};

for row in sort:
    if row[3] == current_region:
        # Append x, y, z to appropiate list
        x.append(row[0]);
        y.append(row[1]);
        z.append(row[2]);
    else:
        # Store in centroids dictionary with key current_region the average x, y, z position.
        # Also store the number of points.
        centroids[current_region] = [np.average(x), np.average(y), np.average(z), len(x)];
        
        # Increment i, change current_region
        i = i + 1;
        current_region = sorted_regions[i]
        
        # Set x, y, z to new row values;
        x = [row[0]];
        y = [row[1]];
        z = [row[2]];

# Store last region averages also!
centroids[current_region] = [np.average(x), np.average(y), np.average(z), len(x)];

In [27]:
output_path = 'output/Aut1367_centroids_by_size.html'

In [ ]:
current_palette = sns.color_palette("husl", len(sorted_regions))
i = 0;

data = [];
for key in sorted_regions:
    current_values_list = centroids[key];
    
    tmp_col = current_palette[i];
    tmp_col_lit = 'rgb' + str(tmp_col);

    trace_scatter = Scatter3d(
        x = [current_values_list[0]],
        y = [current_values_list[1]],
        z = [current_values_list[2]],
        mode = 'markers',
        name = key,
        marker = dict(
            size= np.divide(float(current_values_list[3]), 10000) * 1000,
            color = tmp_col_lit,     # set color to an array/list of desired values
            colorscale = 'Viridis',  # choose a colorscale
            opacity = 0.5
        )
    )

    data.append(trace_scatter)
    i = i + 1;
        
layout = Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    paper_bgcolor='rgb(0,0,0)',
    plot_bgcolor='rgb(0,0,0)'
)

fig = Figure(data=data, layout=layout)

if not os.path.exists('output'):
    os.makedirs('output')

if output_path != None:
    plotly.offline.plot(fig, filename=output_path)

In [44]:
## For s275 to ara3:
points_path = 's275_to_ara3.csv'
output_path = 'output/s275_centroids_by_size.html'
p, uniq = a3.get_regions(points_path, "atlas/ara3_annotation.nii", "points/" + 's275_to_ara3_run' + "_regions.csv");

points_path = 'points/s275_to_ara3_run_regions.csv'
generate_scaled_centroids_graph('s275', points_path, uniq, output_path = output_path)

10050
num unique regions: 409
[0, 2, 4, 6, 9, 10, 15, 17, 19, 23, 26, 27, 33, 35, 36, 38, 42, 50, 56, 59, 62, 63, 64, 67, 68, 72, 73, 74, 81, 84, 88, 100, 113, 117, 118, 120, 122, 125, 126, 127, 128, 129, 131, 132, 133, 140, 146, 148, 149, 155, 156, 158, 159, 163, 170, 171, 173, 178, 180, 181, 186, 187, 188, 189, 190, 194, 196, 197, 201, 204, 210, 211, 214, 215, 218, 223, 226, 233, 243, 246, 250, 252, 255, 257, 258, 260, 262, 263, 266, 271, 272, 274, 278, 286, 287, 288, 289, 296, 298, 301, 303, 304, 305, 310, 311, 313, 314, 319, 320, 321, 326, 328, 333, 335, 338, 342, 344, 347, 351, 362, 363, 364, 366, 368, 374, 377, 381, 382, 397, 403, 412, 414, 421, 422, 423, 430, 434, 436, 440, 442, 443, 448, 449, 450, 451, 452, 463, 10704, 466, 469, 470, 475, 477, 478, 482, 483, 484, 491, 494, 502, 503, 507, 510, 511, 515, 520, 523, 525, 530, 531, 536, 542, 549, 556, 558, 564, 565, 566, 573, 574, 575, 577, 580, 581, 582, 583, 587, 588, 589, 590, 591, 593, 595, 596, 598, 599, 600, 601, 603, 608, 609

In [43]:
def generate_scaled_centroids_graph(token, points_path, unique_list, output_path=None):
    """
    Generates the plotly centroid html file with proper scaling of the centroid determined by the number of bright spots from the csv file.
    :param points_path: Filepath to points csv.
    :param output_path: Filepath for where to save output html.
    :param resolution: Resolution for spacing (see openConnecto.me spacing)
    :return:
    """
    
    # Type in the path to your csv file here
    thedata = None
    thedata = np.genfromtxt(points_path,
        delimiter=',', dtype='int', usecols = (0,1,2,4), names=['a','b','c', 'region'])
    
    # Save the names of the regions
    """
    Load the CSV of the ARA with CCF v3: in order to generate this we use the ARA API.
    We can download a csv using the following URL:
    http://api.brain-map.org/api/v2/data/query.csv?criteria=model::Structure,rma::criteria,[ontology_id$eq1],rma::options[order$eq%27structures.graph_order%27][num_rows$eqall]
    
    Note the change of ontology_id$eq27 to ontology_id$eq1 to get the brain atlas.
    """
    ccf_txt = 'natureCCFOhedited.csv'

    ccf = {}
    with open(ccf_txt, 'rU') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            # row[0] is ccf atlas index, row[4] is string of full name
            ccf[row[0]] = row[4];
    
    # Sort the csv file by the last column (by order = 'region') using ndarray.sort
    sort = np.sort(thedata, order = 'region');
    
    # Save the unique counts
    unique = [];

    for l in sort:
        unique.append(l[3])

    uniqueNP = np.asarray(unique)
    allUnique = np.unique(uniqueNP)
    numRegionsA = len(allUnique)
    
    """
    First we download annotation ontology from Allen Brain Atlas API.
    It returns a JSON tree in which larger parent structures are divided into smaller children regions.
    For example the "corpus callosum" parent is has children "corpus callosum, anterior forceps", "genu of corpus callosum", "corpus callosum, body", etc
    """

    url = "http://api.brain-map.org/api/v2/structure_graph_download/1.json"
    jsonRaw = requests.get(url).content
    jsonDict = json.loads(jsonRaw)

    """
    Next we collect the names and ids of all of the regions.
    Since our json data is a tree we can walk through it in arecursive manner.
    Thus starting from the root...
    """
    root = jsonDict['msg'][0]
    """
    ...we define a recursive function ...
    """

    leafList = []

    def getChildrenNames(parent, childrenNames={}):
        if len(parent['children']) == 0:
            leafList.append(parent['id'])

        for childIndex in range(len(parent['children'])):
            child = parent['children'][childIndex]
            childrenNames[child['id']] = child['name']

            childrenNames = getChildrenNames(child, childrenNames)
        return childrenNames

    """
    ... and collect all of the region names in a dictionary with the "id" field as keys.
    """

    regionDict = getChildrenNames(root)

    # Store most specific data
    specificRegions = [];

    for l in sort:
        if l[3] in leafList:
            specificRegions.append(l)

    # Find all unique regions of brightest points (new)
    uniqueFromSpecific = [];

    for l in specificRegions:
        uniqueFromSpecific.append(l[3])

    # Convert to numpy and save specific lengths
    uniqueSpecificNP = np.asarray(uniqueFromSpecific)
    allUniqueSpecific = np.unique(uniqueSpecificNP)
    numRegionsASpecific = len(allUniqueSpecific)
    specificRegionsNP = np.asarray(specificRegions)

    print "Total number of unique ID's:"
    print numRegionsASpecific  ## number of regions

    # Save a copy of this sorted/specific csv in the points folder
    if not os.path.exists('points'):
        os.makedirs('points')
        
    np.savetxt('points/' + str(token) + '_regions_sorted.csv', sort, fmt='%d', delimiter=',')
    np.savetxt('points/' + str(token) + '_regions_sorted_specific.csv', specificRegionsNP, fmt='%d', delimiter=',')
    
    # Find the centroids of each region
    sorted_regions = np.sort(unique_list);

    current_region = sorted_regions[0];
    i = 0;
    x = [];
    y = [];
    z = [];
    centroids = {};

    for row in specificRegionsNP:
        if row[3] == current_region:
            # Append x, y, z to appropiate list
            x.append(row[0]);
            y.append(row[1]);
            z.append(row[2]);
        else:
            # Store in centroids dictionary with key current_region the average x, y, z position.
            # Also store the number of points.
            centroids[current_region] = [np.average(x), np.average(y), np.average(z), len(x)];

            # Increment i, change current_region
            i = i + 1;
            current_region = sorted_regions[i]

            # Set x, y, z to new row values;
            x = [row[0]];
            y = [row[1]];
            z = [row[2]];

    # Store last region averages also!
    centroids[current_region] = [np.average(x), np.average(y), np.average(z), len(x)];
    
    trace = [];
    for l in specificRegionsNP:
        if str(l[3]) in ccf.keys():
            trace = ccf[str(l[3])]
    
    # Set color pallete to number of specific regions
    current_palette = sns.color_palette("husl", numRegionsA)
    i = 0;

    data = [];
    
    for key in centroids.keys():
        if str(key) not in ccf.keys():
            print key
        else:
            current_values_list = centroids[key];

            tmp_col = current_palette[i];
            tmp_col_lit = 'rgb' + str(tmp_col);

            trace_scatter = Scatter3d(
                x = [current_values_list[0]],
                y = [current_values_list[1]],
                z = [current_values_list[2]],
                mode = 'markers',
                name = ccf[str(key)],
                marker = dict(
                    size= np.divide(float(current_values_list[3]), 10000) * 500,
                    color = tmp_col_lit,     # set color to an array/list of desired values
                    colorscale = 'Viridis',  # choose a colorscale
                    opacity = 0.5
                )
            )

            data.append(trace_scatter)
        i = i + 1;

    layout = Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        ),
        paper_bgcolor='rgb(0,0,0)',
        plot_bgcolor='rgb(0,0,0)'
    )

    fig = Figure(data=data, layout=layout)

    if not os.path.exists('output'):
        os.makedirs('output')

    if output_path != None:
        plotly.offline.plot(fig, filename=output_path)

In [23]:
ccf_txt = 'natureCCFOhedited.csv'

ccf = {}
with open(ccf_txt, 'rU') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        # row[0] is ccf atlas index, row[4] is string of full name
        ccf[row[0]] = row[4];

In [24]:
print ccf['312782562']

Anterior area, layer 5


In [27]:
reload(a3)

<module 'analysis3' from 'analysis3.py'>

In [29]:
a3.generate_region_graph('s275_to_ara3_test', points_path, output_path = 'test.html')

Total number of unique ID's:
343
Done counting!


({'data': [{'marker': {'color': 'rgb(0.9677975592919913, 0.44127456009157356, 0.5358103155058701)',
     'colorscale': 'Viridis',
     'opacity': 0.5,
     'size': 3},
    'mode': 'markers',
    'name': 'Primary somatosensory area, mouth, layer 6b',
    'type': 'scatter3d',
    'x': [849],
    'y': [294],
    'z': [520]},
   {'marker': {'color': 'rgb(0.9679182505914964, 0.44217197691188087, 0.5279992503747023)',
     'colorscale': 'Viridis',
     'opacity': 0.5,
     'size': 3},
    'mode': 'markers',
    'name': 'internal capsule',
    'type': 'scatter3d',
    'x': [715,
     729,
     739,
     740,
     742,
     744,
     748,
     750,
     759,
     761,
     763,
     767,
     771,
     772,
     772,
     776,
     776,
     777,
     779,
     782,
     783,
     784,
     785,
     785,
     793,
     797,
     798,
     798,
     800,
     800,
     802,
     802,
     805,
     806,
     807,
     811,
     813,
     821,
     822,
     825,
     830,
     832,
     837,
 